# Tutorial 3: Multi-turn Prompting

In this tutorial, we'll explore RadPrompter's support for multi-turn prompting. This allows for more complex interactions with the model, where the model's output from one turn can inform the prompt for the next turn.

## Prompt

As usual, we start by importing the `Prompt` class and loading our TOML file:

In [1]:
with open("./03_Multiturn-Prompting.toml", "r") as f:
    lines = f.readlines()
print("".join(lines))

[METADATA]
version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]
system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured. Here is the report:
<report>
{{report}}
</report>
"""

user_prompt_cot = """
I want you to extract the following data element from the report: 'Pulmonary Embolism'
Here are your options and you can explicitly use one of these:
  - `Present`
  - `Absent`
Hint: "Indicate `Present` if the report explicitly mentions the patient has pulmonary embolism in their CT scan. Indicate `Absent` if pulmonary embolism is not seen or if a previously observed pulmonary embolism is mentioned as resolved.
After you provide the data element, I will ask you to provide an explanation and then the final answer.
Now give your initial answer. Then 

In this TOML file, we've defined our prompt components in the `[PROMPTS]` section as before. The key difference is in the `[CONSTRUCTOR]` section:

```toml
[CONSTRUCTOR]
system = "rdp(system_prompt)"
user = ["rdp(user_prompt_intro + user_prompt_cot)", "rdp(user_prompt_format)"]
stop_tags = ["</answer>", "}"]
```

Instead of a single user prompt, we now have a list of two prompts. The first prompt asks the model to extract a data element from the report and provide an explanation, just like in the previous tutorial. The second prompt then asks the model to format its response as JSON.

We've also updated the `stop_tags` to include both the `</answer>` tag from the first prompt and the `}` character that will end the JSON object in the second prompt.

Let's create our prompt object:

In [2]:
from radprompter import Prompt

prompt = Prompt("./03_Multiturn-Prompting.toml")
prompt

## Client & Engine

We'll use the `vLLMClient` and `RadPrompter` engine as in previous tutorials:

In [3]:
from radprompter import RadPrompter, vLLMClient

client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_3.csv",
    concurrency=2,
)

And we run it on our sample reports:

In [4]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

Processing items: 100%|██████████| 3/3 [00:13<00:00,  4.47s/it]


The engine will process each report using our multi-turn prompt and save the results to `output_tutorial_3.csv`.

In [5]:
import pandas as pd

df = pd.read_csv("output_tutorial_3.csv", index_col='index')
df

,default_response_0,default_response_1,report,file_name
index,,,,
0,<answer>\n<Present</Present>\n_initial_answer_...,"{\n ""answer"": {\n ""data_element"": ""P...",Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
1,<answer>\n<Present>\n<initial_answer>\nBased o...,"{\n ""answer"": {\n ""Present"",\n ...",Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
2,<answer>\n<initial_answer>\nPresent\n</initial...,"{\n ""answer"": {\n ""initial_answer"": ...",Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


As you can see, the `default_response_0` and `default_response_1` columns hold the LLM response to first and second question, respectively. Please note that the `stop_tag` is not returned in the LLM response (see Tutorial 02).

Finally, we save the log:

In [6]:
engine.save_log("log_tutorial_3.log")

with open("log_tutorial_3.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.0.8
Model: meta-llama/Meta-Llama-3-8B-Instruct
Prompt TOML: /Users/bardiakhosravi/Desktop/GitHub/RadPrompter/tutorials/03_Multiturn-Prompting/03_Multiturn-Prompting.toml
Prompt Version: 0.1
Prompt Hash: 06f66066098813d66e786cbc2d86e6fa
Concurrency Factor: 2
Start Time: 2024-05-19 15:10:58
End Time: 2024-05-19 15:11:11
Duration: 13.0
Number of Items: 3
Average Processing Time: 4.333333333333333


-------------------- *** - Prompt Content - *** --------------------
[METADATA]
version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]
system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured. Here is the report:
<report>
{{report}}
</report>
"""

user_prompt_cot = """
I want you to extract the following data element from the repor

This tutorial introduces the concept of multi-turn prompting, where the `user` parameter in the `[CONSTRUCTOR]` section is a list of prompts instead of a single prompt. The model's output from each prompt is used as context for the next prompt, allowing for more complex interactions.

The provided TOML file shows an example of a two-turn prompt, where the first turn asks the model to extract a data element and provide an explanation, and the second turn asks the model to format this response as JSON.